In [ ]:
# # # 필요 패키지 설치
# !pip install bertopic
# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git
# !pip install tf-keras

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import json
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from eunjeon import Mecab
from sklearn.model_selection import train_test_split
import tqdm
import re
import datetime
import os
import glob
from pykospacing import Spacing


In [ ]:
data_dir = 'data/real_data'

file_paths = glob.glob(os.path.join(data_dir, '*.xlsx'))
dataset = pd.DataFrame()

for file in file_paths:
      # 데이터 불러오기
      data = pd.read_excel(file)
      # 데이터프레임 합치기
      dataset = pd.concat([dataset, data], ignore_index=True)

dataset.head()

In [ ]:
# 중복 리뷰 삭제
def remove_duplicates(dataset):

  print(f"중복 제거 전 데이터 크기 : {len(dataset)}")
  dataset.drop_duplicates(subset=['리뷰'], inplace=True) 
  print(f"중복 제거 후 전체 데이터 크기 : {len(dataset)}")

  return dataset

In [ ]:
# 한글 제외한 문자 제거
def remove_not_korean(dataset):
  review_removed = list(map(lambda review: re.sub('[^가-힣 ]', '', review), dataset['리뷰']))
  dataset['리뷰'] = review_removed

  return dataset

In [ ]:
dataset = remove_not_korean(dataset)
dataset.head()

In [ ]:
# 띄어쓰기 변환기
def process_spacing(dataset):
  spacing = Spacing() 
  spacing_review = list()
  for review in tqdm.tqdm(dataset['리뷰']):
    spacing_review.append(spacing(review))
  
  dataset['리뷰'] = spacing_review

  return dataset
    

In [ ]:
dataset = process_spacing(dataset)
dataset.head()

In [ ]:
# 긍정적 리뷰(평점 5점 만점 기준 4, 5점)
review_pos = dataset[(dataset['별점'] == 4) | (dataset['별점'] == 5)]['리뷰']
# 부정적 리뷰(평점 5점 만점 기준 1, 2, 3점)
review_neg = dataset[(dataset['별점'] == 1) | (dataset['별점'] == 2) | (dataset['리뷰'] == 3)]['리뷰']

review_pos.head()

In [ ]:
review_pos = pd.DataFrame(review_pos, columns=['리뷰'])
review_neg = pd.DataFrame(review_neg, columns=['리뷰'])

review_pos.head()

In [ ]:
print(f"긍정적 리뷰 개수 : {len(review_pos)}")
print(f"부정적 리뷰 개수 : {len(review_neg)}")

In [ ]:
#치환 리스트
replace_list = pd.read_excel('data/preprocess_list/replace_list.xlsx')
replace_list.head()

In [ ]:
# 단어 치환 리스트 적용
def replace_word(dataset):
    review_replaced_list = []
    for review in tqdm.tqdm(dataset['리뷰']):
        review = ''.join(review)  # 리스트의 요소들을 공백으로 연결
        review = str(review)
        for before, after in zip(replace_list['before_replacement'], replace_list['after_replacement']):
            review = review.replace(before, after)  # 각 치환 적용
        review_replaced_list.append(review)  # 최종적으로 치환된 리뷰를 리스트에 추가

    dataset['치환된 리뷰'] = review_replaced_list
    
    len(dataset['리뷰'])
    len(review_replaced_list)
    
    dataset['치환된 리뷰'] = review_replaced_list

    return dataset


In [ ]:
review_pos = replace_word(review_pos)
review_neg = replace_word(review_neg)

In [ ]:
review_pos.head(100)

In [ ]:
def tokenizer(dataset):
    result = []
    for review in dataset['치환된 리뷰']:
        #nouns : 명사 추출, pos : 품사 부착, morphs : 형태소 추출
        result.append(mecab.nouns(str(review)))

    print(len(dataset['치환된 리뷰']))
    print(len(result))
    dataset['토큰'] = result
    return dataset

In [ ]:
# 토큰화
mecab = Mecab()

review_tokenized_pos = tokenizer(review_pos)
review_tokenized_neg = tokenizer(review_neg)

In [ ]:
review_tokenized_pos.head()

In [ ]:
print(f'긍정적 리뷰 형태소 개수 : {sum(len(sublist) for sublist in review_tokenized_pos["토큰"])}')
print(f'긍정적 리뷰 형태소 개수 : {sum(len(sublist) for sublist in review_tokenized_neg["토큰"])}')

In [ ]:
#불용어 리스트 정의
stopword_list = pd.read_excel('data/preprocess_list/stopword_list.xlsx')
stopword_list.head()

In [ ]:
#한 글자 토픽 정의
one_char_keyword = pd.read_excel('data/preprocess_list/one_char_list.xlsx')
one_char_keyword.head()

In [ ]:
# 욕설 리스트 정의
abuse_list = pd.read_excel('data/preprocess_list/fword_list.xlsx')
abuse_list.head()

In [ ]:
# 불용어 제거 및 한 글자 토픽 제외한 한 글자 단어 삭제
def remove_stopword(dataset):
    review_removed_stopword = []
    for tokens in dataset['토큰']:
        token_removed_stopword = []
        for token in tokens:
            # 토큰이 욕설 리스트에 없는 경우
            if token not in list(abuse_list['fword']):
                # 토큰의 글자 수가 2글자 이상인 경우
                if 1 < len(token):
                    # 토큰이 불용어가 아닌 경우만 분석용 리뷰 데이터로 포함
                    if token not in list(stopword_list['stopword']):
                        token_removed_stopword.append(token)
                # 토큰의 글자 수가 1글자인 경우
                else:
                    # 1글자 키워드에 포함되는 경우만 분석용 리뷰 데이터로 포함
                    if token in list(one_char_keyword['one_char_keyword']):
                        token_removed_stopword.append(token)
            
        review_removed_stopword.append(token_removed_stopword)
    
    dataset['불용어 제거 후 토큰'] = review_removed_stopword
    
    return dataset

In [ ]:
# 긍정 리뷰, 부정 리뷰 각각에 욕설 제거, 불용어 제거, 한 글자 키워드 제외 한 글자 단어 제거 적용
review_removed_stopword_pos = remove_stopword(review_tokenized_pos)
review_removed_stopword_neg = remove_stopword(review_tokenized_neg)

In [ ]:
review_removed_stopword_pos.head()

In [ ]:
print(f'단어 필터링 후 긍정적 리뷰 형태소 개수 : {sum(len(sublist) for sublist in review_removed_stopword_pos["불용어 제거 후 토큰"])}')
print(f'단어 필터링 후 부정적 리뷰 형태소 개수 : {sum(len(sublist) for sublist in review_removed_stopword_neg["불용어 제거 후 토큰"])}')
print(len(review_removed_stopword_pos["불용어 제거 후 토큰"]))

In [ ]:
#불용어 제거 후 토큰이 없는 경우, 해당 데이터 행 제거
review_removed_stopword_pos = review_removed_stopword_pos[review_removed_stopword_pos['불용어 제거 후 토큰'] != '[]']
review_removed_stopword_neg = review_removed_stopword_neg[review_removed_stopword_neg['불용어 제거 후 토큰'] != '[]']

review_removed_stopword_pos.head()

In [ ]:
# 리스트를 문자열로 결합
review_removed_stopword_pos["학습용 데이터"] = [' '.join(tokens) for tokens in review_removed_stopword_pos["불용어 제거 후 토큰"]]
review_removed_stopword_neg["학습용 데이터"] = [' '.join(tokens) for tokens in review_removed_stopword_neg["불용어 제거 후 토큰"]]

In [ ]:
review_removed_stopword_pos.head()

In [ ]:
# 전처리 후 데이터를 excel 파일로 export
df1 = pd.DataFrame(review_removed_stopword_pos, columns=['리뷰','치환된 리뷰','토큰','불용어 제거 후 토큰','학습용 데이터'])
df2 = pd.DataFrame(review_removed_stopword_neg, columns=['리뷰','치환된 리뷰','토큰','불용어 제거 후 토큰','학습용 데이터'])

df1.to_excel("data/preprocessed_data/processed_pos_data.xlsx", index = False, engine='openpyxl')
df2.to_excel("data/preprocessed_data/processed_neg_data.xlsx", index = False, engine='openpyxl')

In [ ]:
# 전처리 후 데이터 load
review_removed_stopword_pos = pd.read_excel('data/preprocessed_data/processed_pos_data.xlsx')
review_removed_stopword_neg = pd.read_excel('data/preprocessed_data/processed_neg_data.xlsx')

In [ ]:
# 학습 데이터에 null 있는지 체크
print(review_removed_stopword_pos['학습용 데이터'].isnull().sum())
print(review_removed_stopword_neg['학습용 데이터'].isnull().sum())

In [ ]:
import umap
import hdbscan
from bertopic.representation import KeyBERTInspired

# 한국어 BERT 임베딩 모델 로드
embedding_model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens')
umap_model = umap.UMAP(n_neighbors = 10, min_dist = 0.1, n_components=4, random_state=42, metric='euclidean')
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size = 6, metric = 'euclidean', cluster_selection_method='eom', prediction_data=True)
representation_model = KeyBERTInspired()
zeroshot_topic_list = ['스토리','힐링','재미','캐릭터','더빙','시스템','컨텐츠']

hyperparams = {
     'top_n_words' : 10,
     'nr_topics' : 20,
     'n_gram_range' : (1, 1),
     'min_topic_size' : 10,
     'calculate_probabilities' : True,
     'zeroshot_topic_list' : zeroshot_topic_list,
     'zeroshot_min_similarity' : 85,
}

# BERTopic 모델 생성 및 학습
topic_model = BERTopic(embedding_model=embedding_model, umap_model=umap_model, representation_model=representation_model, **hyperparams)


In [ ]:
# BERTopic 모델 적용
pos_topics, pos_probabilities = topic_model.fit_transform(review_removed_stopword_pos['학습용 데이터'])
neg_topics, neg_probabilities = topic_model.fit_transform(review_removed_stopword_neg['학습용 데이터'])

In [ ]:
# 원본 리뷰 + 토픽 저장
review_removed_stopword_pos['토픽'] = pos_topics
pos_max_probabilities = np.max(pos_probabilities, axis=1)
review_removed_stopword_pos['토픽에 속할 확률'] = pos_max_probabilities

review_removed_stopword_neg['토픽'] = neg_topics
neg_max_probabilities = np.max(neg_probabilities, axis=1)
review_removed_stopword_neg['토픽에 속할 확률'] = neg_max_probabilities

In [ ]:
# 모델 매개변수 저장
model_name = 'v4'
review_removed_stopword_pos.to_excel("/result/result/after_train_data.xlsx", index = False, engine='openpyxl')

with open(f"result/params/{model_name}.json", "w") as f:
    json.dump(hyperparams, f)
print(f"모델 매개변수가 {model_name}.json 파일로 저장되었습니다.")

topic_model.save(f"result/models/model_{model_name}")
print(f"모델이 {model_name}_model 파일로 저장되었습니다.")

topic_info_df = topic_model.get_topic_info()
topic_info_df.to_excel(f'result/topics/{model_name}_topic_info.xlsx', index=False)
print(f"토픽 정보가 {model_name}_topic_info.xlsx 파일로 저장되었습니다.")

In [ ]:
topic_model.visualize_topics()	

In [ ]:
topic_model.visualize_barchart()		

In [ ]:
topic_model.visualize_heatmap()

In [ ]:
topic_model.visualize_hierarchy()	

In [ ]:
topic_model.visualize_topics()